In [1]:
import numpy as np
import matplotlib.pyplot as pl
import sympy as sm

import scipy.optimize as sopt
import matplotlib.pyplot as plt

from   mpl_toolkits import mplot3d

import seaborn as sb

In [2]:
def f(s):
    x, y= s
    
    x0,y0= (8, 1)    
    z=  (x-x0)**2 + ((y-y0)/3)**2 
    
    return z

In [3]:
def c1(s):
    x,y= s
    #z= x + y*1e-3 +10  # 模擬 x + 10 > =0
    z= x + y + 6        # x + y - 1 >= 0
    return z

def c2(s):
    x,y= s
    #z= x*1e-3 +y +10   # 模擬 y + 10 > =0
    z= -x + y +6      # -x + 2*y - 2 >= 0
    return z

def c3(s):
    x,y= s
    #z= -(x + 3*y -9)
    #z= x + y + 20
    #z= -x - 3*y +30
    ## 試試看2次函數當作constraints
    z= -(x/2)**2 - y +10  # -x**2 - y +10 >=0
    return z

Constraints= [
    {'fun':c1, 'type':'ineq'},
    {'fun':c2, 'type':'ineq'},
    {'fun':c3, 'type':'ineq'},
]

xyBounds= [(-5, 5),  # xmin, xmax
           (-5, 5)   # ymin, ymax
           ]
def ryOptimization(
        f= f, 
        c1= c1, 
        c2= c2, 
        c3= c3, 
        xyBounds= xyBounds):

    #
    # Objective function s.t. Constraints
    #
    x,y= sm.symbols('x,y')
    
    print(f'Objective function: f()={f([x,y])}')
    q= f([x,y])
    display(q)
    
    print('Constraints: c() >= 0')
    print(f'c()= {c1([x,y])},{c2([x,y])},{c3([x,y])}')
    
    print('xyBounds:  [xmin, xmax] [ymin, ymax]')
    print(f'{xyBounds}')
    
    q= c1([x,y])
    display(q)
    q= c2([x,y])
    display(q)
    q= c3([x,y])
    display(q)
    
    x0= np.random.random(2)*10-5
    
    opt= sopt.minimize(
        f,
        x0= x0,
        constraints= [
            {'fun':c1, 'type': 'ineq'},
            {'fun':c2, 'type': 'ineq'},
            {'fun':c3, 'type': 'ineq'}        
            ],
        #bounds= [(None, None),
        #         (-3, None)
        #         #(None, -3)]
        bounds= xyBounds                
    )
    print(f'optimum= \n{opt}')
    
    
    #
    # plot objective function
    #
    x,y= sm.symbols('x,y')
    #f([x,y])
    '''
    fg, ax= ryPlotGradientMap(
        f([x,y]), 
        xrange= (-10,10), 
        yrange= (-10,10)
    )
    '''
    fg, ax, _, _, _=  ryPlot3d(
        f([x,y]), 
        xrange= (-10,10), 
        yrange= (-10,10))
    
    
    #
    # plot constraints 
    #
    
    xx= np.linspace(-10,10,1001)
    
    '''
    yy1= 3*xx+6
    yy2= (-xx+4)/2
    yy3= xx*0 -3
    '''
    
    # 上面3行 constraints 
    # 如何自動從 c1, c2, c3 撈出來？？
    
    y1= sm.lambdify(x,
            sm.solve(
                c1([x,y]),
                y)[0]
            )
    y2= sm.lambdify(x,
            sm.solve(
                c2([x,y]),
                y)[0]
            )
    y3= sm.lambdify(x,
            sm.solve(
                c3([x,y]),
                y)[0]
            )
    yy1= y1(xx)
    yy2= y2(xx)
    yy3= y3(xx)
    #-----------------------
    
    ax.plot(xx,yy1,'r',
            xx,yy2,'g',
            xx,yy3,'b',
            linestyle='--'
           )
    #
    # 畫 最佳點 opt.x 
    #
    
    ax.scatter3D(
        xs= [opt.x[0]]*2, 
        ys= [opt.x[1]]*2, 
        zs= [0, opt.fun],
        color= 'magenta',
        marker= 's'    
    )
    
    
    ax.plot3D(
        xs= [opt.x[0]]*2, 
        ys= [opt.x[1]]*2, 
        zs= [0, opt.fun],
        color= 'magenta',
        #marker= '.',
        linestyle= ':'
    )
    
    
    
    #
    # 畫坐標軸 (x軸,y軸) 
    #
    ax.plot(xx,  xx*0,'k--',
            xx*0,xx,  'k--',
           )
    
    #
    #  畫 xyBounds (平行x軸,y軸) 
    #
    
    ax.plot(xx,  xyBounds[1][0]*np.ones_like(xx),'c--',
            xx,  xyBounds[1][1]*np.ones_like(xx),'c--',
            xyBounds[0][0]*np.ones_like(xx), xx, 'c--',
            xyBounds[0][1]*np.ones_like(xx), xx, 'c--',
           )
    
    
    
    #
    # 設定 2軸 顯示範圍
    #
    ax.set_xlim([-10,10])
    ax.set_ylim([-10,10])
    
    
    #
    # 再加碼 ，把 constraints 升至 立體的曲面 
    #
    
    xx= np.linspace(-10,10,1001)
    
    #yy1= 3*xx+6
    #yy2= (-xx+4)/2
    #yy3= xx*0 -3
    
    zz1= f([xx,yy1])
    zz2= f([xx,yy2])
    zz3= f([xx,yy3])
    
    ax.plot3D(xx,yy1,zz1, color='r', linestyle='-')
    ax.plot3D(xx,yy2,zz2, color='g', linestyle='-')
    ax.plot3D(xx,yy3,zz3, color='b', linestyle='-')
    
    pl.show()

In [4]:
def ryPlot3d_ver0_(z, xrange=(-3,+3), yrange=(-3,+3), kwargs= None):
    
    if kwargs == None:
        kwargs={'xlabel': 'x', 'ylabel': 'y'}
    
    xmin, xmax= xrange
    ymin, ymax= yrange
    
    xm,ym= np.meshgrid(
        np.linspace(xmin,xmax,101),
        np.linspace(ymin,ymax,101)
    )
    zm= sm.lambdify((x,y),z)(xm,ym)

    fg= pl.figure()
    ax= pl.axes(projection='3d',
                title= f'z= ${sm.latex(z)}$',
                **kwargs)
    ax.contour3D(xm, ym, zm, 100,
                 cmap= 'rainbow')
    return fg, ax    
    
# 進入真正的深水區
# 運用 ax.quiver() 結合 Gradient Vector ，
# 把 2D 函數 的梯度圖畫出。

def ryPlot3d(z, xrange=(-3,+3), yrange=(-3,+3), kwargs= None):
    
    if kwargs == None:
        kwargs={'xlabel': 'x', 'ylabel': 'y'}
    
    xmin, xmax= xrange
    ymin, ymax= yrange
    
    xrange= np.linspace(xmin,xmax,101)
    yrange= np.linspace(ymin,ymax,101)
    xm,ym=  np.meshgrid(xrange,yrange)
    
    zm= sm.lambdify((x,y),z)(xm,ym)

    fg= pl.figure()
    ax= pl.axes(projection='3d',
                title= f'z= ${sm.latex(z)}$',
                **kwargs)
    ax.contour3D(xm, ym, zm, 100,
                 cmap= 'rainbow')
    return fg, ax, zm, xrange, yrange

def ryPlotGradientMap(z,
                      xrange=(-3,+3), 
                      yrange=(-3,+3),
                      downSampleFactor= 4, 
                      lengthFactor= .1):
    
    fg, ax, zm, xrange, yrange=  ryPlot3d(z, 
                                          xrange= xrange, 
                                          yrange=yrange)
    
    z10= z.diff(x,1,y,0)
    z01= z.diff(x,0,y,1)

    #downSampleFactor= 2
    xm, ym, zm0= np.meshgrid(
        xrange[::downSampleFactor],
        yrange[::downSampleFactor],
        np.linspace(np.min(zm), np.min(zm)+1, 1)
        )

    #dz_dx= 2*x0
    #dz_dy= 2*y0

    dz_dx_m= sm.lambdify((x,y),z10)(xm,ym)
    dz_dy_m= sm.lambdify((x,y),z01)(xm,ym)

    u= dz_dx_m
    v= dz_dy_m
    w= np.zeros_like(xm)
    ax.quiver(xm, ym, zm0, 
              u, v, w, 
              length= lengthFactor, #*np.max([u,v]), 
              color = 'gray')
    
    return fg, ax


# 封裝測試一下...，允許輸入不同函數
# 加碼練習 subplot()
# 加碼觀察 cumsum()

def ryHistogram_demo(func= None):
    
    # 在時間軸上 不同的訊號，
    # 使用 np.histogram() 把它們視為相同的機率分布！
    t= np.linspace(-10,10,1001)
    #t= np.arange(-10,10)
    
    if func==None:
        func= np.sin
        
    X= func(t) #t**2
    
    X1= np.sort(X)
    X2= X.copy()
    np.random.shuffle(X2)  # 這個 .shuffle() 每次結果都不同！！
    
    
    # 針對離散型，也許 nbins 要精細一點計算
    nbins= len(set(X))
    nbins= min(nbins, 20)
    
    f, x= np.histogram(X, bins= nbins)
    f= np.insert(f,0,0)  
    # 上面這一行是 ry 特殊補丁，
    # .histogram() 做出的 f, x 長度不同，影響後面程式碼的簡潔！
    f1, x1= np.histogram(X1, bins= nbins)
    f1= np.insert(f1,0,0)  
    f2, x2= np.histogram(X2, bins= nbins)
    f2= np.insert(f2,0,0)
    t, X, X1, X2
    x, f, x1, f1, x2, f2
    
    # 加碼觀察 cumsum
    F= np.cumsum(f)
    
    #fg1= pl.figure('fg1')
    #ax= pl.axes()
    
    fg, ax = plt.subplots(2,2,figsize=(10,5))
    
    ax[0,0].plot(t,X,'ro',t,X1,'gx',t,X2,'b.')
    ax[0,0].set_xlabel('t')
    ax[0,0].set_ylabel('X, X1, X2')
    ax[0,0].legend(['X','X1','X2'])
    
    #'''
    #fg2= pl.figure('fg2')
    #ax2= pl.axes()
    ax[0,1].plot(f,x,'ro--',f1,x1,'gx-.',f2,x2,'b.-',F,x,'cs--')
    ax[0,1].set_xlabel('f, f1, f2, F')
    ax[0,1].set_ylabel('x')
    ax[0,1].legend(['f','f1','f2','F'])
    #'''
    #fg3= pl.figure('fg3')
    #ax3= pl.axes()
    ax[1,0].plot(x,f,'ro--',x1,f1,'gx-',x2,f2,'b.-', x,F,'cs--')
    ax[1,0].set_ylabel('f, f1, f2, F')
    ax[1,0].set_xlabel('x')
    ax[1,0].legend(['f','f1','f2','F'])
    
    # 加碼觀察 cumsum
    # 本想簡單達成
    # 但 加 legend() 出問題，只好把以下幾行混進上面程式碼中，殘念！！！
    
    '''
    F= np.cumsum(f)
    ax[1,0].plot(x,F,'cs-')
    #ax[1,0].legend(['F'])
    ax[0,1].plot(F,x,'cs-')
    #ax[0,1].legend(['F'])
    '''
 
    
def ryDistribution_demo(distributionType= 'normal', sampleSize= 1000):
    
    #sampleSize= 1000

    t= np.arange(sampleSize)
    
    if distributionType in ['normal','gauss','gaussian']:
        X= np.random.normal(size= sampleSize)
    elif distributionType == 'uniform':
        X= np.random.uniform(0,10, size= sampleSize)
    elif distributionType == 'binomial':
        X= np.random.binomial(n=10, p=.5, size= sampleSize)
    elif distributionType == 'poisson':
        X= np.random.poisson(lam=5, size= sampleSize)
    elif distributionType == 'uniform_d':
        X= np.random.randint(low= 0, high= 10, size= sampleSize)
    else: # 以上皆非時，以下 sin() 取代
        X= np.sin(t/sampleSize*2*np.pi)
        X += np.sin(t/sampleSize*2*np.pi *2)
        X += np.sin(t/sampleSize*2*np.pi *3)
        
    #X= X*X

    fig,ax = plt.subplots(3, 1,figsize=(6,3))
    ax[0].plot(t,X,'r.',alpha=.5)

    X1= X.copy()
    X1.sort()

    ax[0].plot(t,X1,'g.',alpha=.3)

    ax[0].set_xlabel('t')
    ax[0].set_ylabel('X')
    ax[0].legend(['X','X1= X.sort()'])
    ax[0].set_title(f'{distributionType}, X= X(t)')

    ax[1].plot(X,t,'r.',alpha=.5)
    ax[1].plot(X1,t,'g.',alpha=.3)

    ax[1].set_xlabel('X')
    ax[1].set_ylabel('t')
    ax[1].legend(['X','X1= X.sort()'])
    ax[1].set_title('$t= t(X)=X^{-1}(X(t))$, t in vertical, X in horizontal')

    # 針對離散型，也許 nbins 要精細一點計算
    nbins= len(set(X))
    nbins= min(nbins, 20)
    
    f, x= np.histogram(X, bins= nbins)
    x= x[:-1] # 原本的 x 比 f 多 1 點，故在此把它弄掉
    
    xwidth=  (X.max()-X.min())/nbins/2
    
    ax[2].bar(x, f, width= xwidth, color='r', alpha=.5)

    F= f.cumsum()
    ax[2].bar(x, F, width= xwidth, color= 'g', alpha=.3)

    ax[2].set_xlabel('x')
    ax[2].set_ylabel('f, F')
    ax[2].legend(['f','F= f.cumsum()'])
    ax[2].set_title('f= histogram(X)= f(x)')


    plt.show()
    

def ryGetAttr(className):
    q= className
    for n, x in enumerate(dir(q)): 
        if not x.startswith('_'):
            y= getattr(q, x)
            z= y.__doc__
            print(f'''
            {'='*40}
            n= {n}
            x= {x}
            y= {y}
            {'-'*40}
            z= {z}
            ''')

In [5]:
ryOptimization(f,c1,c2,c3, xyBounds)

Objective function: f()=(x - 8)**2 + (y/3 - 1/3)**2


(x - 8)**2 + (y/3 - 1/3)**2

Constraints: c() >= 0
c()= x + y + 6,-x + y + 6,-x**2/4 - y + 10
xyBounds:  [xmin, xmax] [ymin, ymax]
[(-5, 5), (-5, 5)]


x + y + 6

-x + y + 6

-x**2/4 - y + 10

optimum= 
     fun: 9.000000000000254
     jac: array([-6.00000000e+00, -2.38418579e-07])
 message: 'Optimization terminated successfully'
    nfev: 12
     nit: 4
    njev: 4
  status: 0
 success: True
       x: array([5.        , 0.99999876])


NameError: name 'x' is not defined